In [1]:
import pandas as pd
import numpy as np

# Define File Paths

In [2]:
ALL_PP = "/Users/angelina/Dropbox (Emotive Computing)/CPS_Gaze_Fixations/utils/All_PP.csv"
SUPPLEMENTAL = "/Users/angelina/Dropbox (Emotive Computing)/CPS_Gaze_Fixations/RQA/Supplemental/"
TEAMSUMMARY = SUPPLEMENTAL + "Summary_Teams_PP_Logs_Lab.csv"
PERSON_BLOCK = SUPPLEMENTAL + "person_block.csv"
OUTPUTDIR = "/Users/angelina/Dropbox (Emotive Computing)/CPS_Gaze_Fixations/RQA/"


# Import Supplemental Files

In [3]:
# Read in the All_PP file which is our source of truth for the complete teams (2 participants) and their PIDs
dfAll_PP = pd.read_csv(ALL_PP)

#  Read in file that contains one row per team --> gold and silver trophies for every block
dfTeamSummary = pd.read_csv(TEAMSUMMARY)

# Read in file that contains one row per person and block --> subjective measures
dfPersonBlock = pd.read_csv(PERSON_BLOCK)

# Informational
print("Number of unique team + blocks in All_PP: ", len(dfAll_PP.groupby(['GROUPID', 'block'])))
print("Number of teams in 'Summary_Teams_PP_Logs_Lab.csv': %d" % (len(dfTeamSummary)))
print("Number of 'participants + blocks' in 'person_block.csv': %d" % (len(dfPersonBlock)))

Number of unique team + blocks in All_PP:  274
Number of teams in 'Summary_Teams_PP_Logs_Lab.csv': 95
Number of 'participants + blocks' in 'person_block.csv': 1112


/var/folders/mk/d16gbl2n2mj9x5gptkp92y5c0000gn/T/ipykernel_7786/2162894968.py:2: DtypeWarning: Columns (21,22,23,24,25,26,30,31,32,36) have mixed types. Specify dtype option on import or set low_memory=False.
  dfAll_PP = pd.read_csv(ALL_PP)


# Calculate Task Score

### Set up DataFrame

In [4]:
dfTaskScores = pd.DataFrame(columns = ['GROUPID', 'block', 'num_gold', 'num_silver', 'task_score'])

### Team Summary File Transformation

In [5]:
# Filter down to columns we want
dfTeamSummary = dfTeamSummary.loc[:, ['Team', \
                                      'WU_Gold_Trophies', 'WU_Silver_Trophies', \
                                      'Golds_Manipulation_Block', 'Golds_Gold_Trophies', 'Golds_Silver_Trophies', \
                                      'Levels_Manipulation_Block', 'Levels_Gold_Trophies', 'Levels_Silver_Trophies']]


# Add School ID
dfTeamSummary['school'] = dfTeamSummary['Team'].str.split("-", expand=True)[1]
dfTeamSummary['schoolID'] = np.where(dfTeamSummary['school'] == "ASU", 10, 20)

# Add Team ID
dfTeamSummary['teamID'] = dfTeamSummary['Team'].str.split("-", expand=True)[2].str.partition("T")[2]

# Add GROUPID
dfTeamSummary['GROUPID'] = dfTeamSummary['schoolID'].astype(str) + dfTeamSummary['teamID'].astype(str)
dfTeamSummary['GROUPID'] = pd.to_numeric(dfTeamSummary['GROUPID'])


### Get trophies for all teams and blocks

In [6]:
# For each team
for i,row in dfTeamSummary.iterrows():
    team_rows = []
    
    # Warmup block
    row_dict = {'GROUPID': row['GROUPID'], 'block': 'Warmup',  \
                    'num_gold': row['WU_Gold_Trophies'], 'num_silver': row['WU_Silver_Trophies']}
    
    team_rows.append(row_dict)
    
    
    # Golds Manipulation - ExpBlock1 or ExpBlock2
    if row['Golds_Manipulation_Block'] == 'ExpBlock1':
        row_dict = {'GROUPID': row['GROUPID'], 'block': 'ExpBlock1',  \
                    'num_gold': row['Golds_Gold_Trophies'], 'num_silver': row['Golds_Silver_Trophies']}
        
        team_rows.append(row_dict)
        
    elif row['Golds_Manipulation_Block'] == 'ExpBlock2':
        row_dict = {'GROUPID': row['GROUPID'], 'block': 'ExpBlock2',  \
                    'num_gold': row['Golds_Gold_Trophies'], 'num_silver': row['Golds_Silver_Trophies']}
        
        team_rows.append(row_dict)
    
    
    # Levels Manipulation - ExpBlock1 or ExpBlock2
    if row['Levels_Manipulation_Block'] == 'ExpBlock1':
        row_dict = {'GROUPID': row['GROUPID'], 'block': 'ExpBlock1',  \
                    'num_gold': row['Levels_Gold_Trophies'], 'num_silver': row['Levels_Silver_Trophies']}
        
        team_rows.append(row_dict)
        
    elif row['Levels_Manipulation_Block'] == 'ExpBlock2':
        row_dict = {'GROUPID': row['GROUPID'], 'block': 'ExpBlock2',  \
                    'num_gold': row['Levels_Gold_Trophies'], 'num_silver': row['Levels_Silver_Trophies']}
        
        team_rows.append(row_dict)
       
    
    # Add team's block + trophies info to Task Scores DataFrame
    dfTaskScores = pd.concat([dfTaskScores, pd.DataFrame(team_rows)], ignore_index=True)
                     

### Calculate Task Score

In [7]:
dfTaskScores['task_score'] = dfTaskScores['num_silver'] + (2 * dfTaskScores['num_gold'])
dfTaskScores

,GROUPID,block,num_gold,num_silver,task_score
0,1038,Warmup,0.0,0.0,0.0
1,1038,ExpBlock2,0.0,1.0,1.0
2,1038,ExpBlock1,1.0,4.0,6.0
3,1039,Warmup,0.0,1.0,1.0
4,1039,ExpBlock1,0.0,0.0,0.0
...,...,...,...,...,...
277,1096,ExpBlock2,1.0,3.0,5.0
278,1096,ExpBlock1,1.0,0.0,2.0
279,1097,Warmup,0.0,1.0,1.0
280,1097,ExpBlock1,3.0,1.0,7.0


# Calculate Subjective Measures

### Person Block File Transformation

In [8]:
# Filter down to columns we want
dfPersonBlock = dfPersonBlock.loc[:, ['team', 'school', 'Block', 'userID', 'CPS_and_ITN_mean', 'Valence']]

# Add School ID
dfPersonBlock['schoolID'] = np.where(dfPersonBlock['school'] == "ASU", 10, 20)

# Add Team ID
dfPersonBlock['teamID'] = dfPersonBlock['team'].str.split("-", expand=True)[1].str.partition("T")[2]

# Add GROUPID
dfPersonBlock['GROUPID'] = dfPersonBlock['schoolID'].astype(str) + dfPersonBlock['teamID'].astype(str)
dfPersonBlock['GROUPID'] = pd.to_numeric(dfPersonBlock['GROUPID'])

dfPersonBlock.rename(columns={"userID": "pid", "Block": "block"}, inplace=True)

dfPersonBlock

,team,school,block,pid,CPS_and_ITN_mean,Valence,schoolID,teamID,GROUPID
0,ASU-T10,ASU,Warmup,CPS2-ASU-T10-PA,-1.037459,4.0,10,10,1010
1,ASU-T10,ASU,ExpBlock1,CPS2-ASU-T10-PA,-1.037459,3.0,10,10,1010
2,ASU-T10,ASU,ExpBlock2,CPS2-ASU-T10-PA,-1.037459,3.0,10,10,1010
3,ASU-T10,ASU,Transfer,CPS2-ASU-T10-PA,-1.037459,3.0,10,10,1010
4,ASU-T10,ASU,Warmup,CPS2-ASU-T10-PB,0.783249,5.0,10,10,1010
...,...,...,...,...,...,...,...,...,...
1107,CU-T71,CU,Transfer,CPS2-CU-T71-PB,0.783249,5.0,20,71,2071
1108,CU-T71,CU,Warmup,CPS2-CU-T71-PC,-0.114509,4.0,20,71,2071
1109,CU-T71,CU,ExpBlock1,CPS2-CU-T71-PC,-0.211003,3.0,20,71,2071
1110,CU-T71,CU,ExpBlock2,CPS2-CU-T71-PC,-0.211003,4.0,20,71,2071


### Person Block File Filtering

In [9]:
# Group by team, block, and participant(GROUPID represents a unique team because it is specific to a school)
all_pp_team_block_pids = dfAll_PP.groupby(['GROUPID', 'block', 'participant_id'], \
                                          as_index=False).size()[['GROUPID', 'block', 'participant_id']]
all_pp_team_block_pids.rename(columns={"participant_id": "pid"}, inplace=True)

# Merge dfPersonBlock w/ All_PP to filter participants we should not process
team_block_pids = pd.merge(all_pp_team_block_pids, dfPersonBlock, on=['GROUPID', 'block', 'pid'], how='inner')

team_block_pids


,GROUPID,block,pid,team,school,CPS_and_ITN_mean,Valence,schoolID,teamID
0,1010,ExpBlock1,CPS2-ASU-T10-PA,ASU-T10,ASU,-1.037459,3.0,10,10
1,1010,ExpBlock1,CPS2-ASU-T10-PB,ASU-T10,ASU,0.783249,5.0,10,10
2,1010,ExpBlock1,CPS2-ASU-T10-PC,ASU-T10,ASU,-0.659882,3.0,10,10
3,1010,ExpBlock2,CPS2-ASU-T10-PA,ASU-T10,ASU,-1.037459,3.0,10,10
4,1010,ExpBlock2,CPS2-ASU-T10-PB,ASU-T10,ASU,0.783249,5.0,10,10
...,...,...,...,...,...,...,...,...,...
817,10104,ExpBlock2,CPS2-ASU-T104-PB,ASU-T104,ASU,0.783249,3.0,10,104
818,10104,ExpBlock2,CPS2-ASU-T104-PC,ASU-T104,ASU,-1.742230,5.0,10,104
819,10104,Warmup,CPS2-ASU-T104-PA,ASU-T104,ASU,-0.915774,1.0,10,104
820,10104,Warmup,CPS2-ASU-T104-PB,ASU-T104,ASU,0.607056,3.0,10,104


### Aggregate Participant- + Block-level Measures to Team- + Block-level

In [10]:
dfSubjMeasures = team_block_pids.loc[:,['GROUPID', 'block', 'CPS_and_ITN_mean', 'Valence']]\
                  .groupby(['GROUPID', 'block'], as_index=False).agg(np.nanmean)

dfSubjMeasures

,GROUPID,block,CPS_and_ITN_mean,Valence
0,1010,ExpBlock1,-0.304697,3.666667
1,1010,ExpBlock2,-0.304697,3.666667
2,1020,ExpBlock1,-0.389995,4.333333
3,1020,ExpBlock2,-0.026413,4.666667
4,1020,Warmup,-0.742380,3.666667
...,...,...,...,...
269,10103,ExpBlock2,-0.026413,3.666667
270,10103,Warmup,-1.405215,3.333333
271,10104,ExpBlock1,-0.531224,3.000000
272,10104,ExpBlock2,-1.023464,4.000000


# Merge Task Scores and Subjective Measures 

In [11]:
dfTeamBlockOutcomes = pd.merge(dfSubjMeasures, dfTaskScores, on=['GROUPID', 'block'], how='inner')

dfTeamBlockOutcomes

,GROUPID,block,CPS_and_ITN_mean,Valence,num_gold,num_silver,task_score
0,1010,ExpBlock1,-0.304697,3.666667,1.0,1.0,3.0
1,1010,ExpBlock2,-0.304697,3.666667,2.0,1.0,5.0
2,1020,ExpBlock1,-0.389995,4.333333,0.0,2.0,2.0
3,1020,ExpBlock2,-0.026413,4.666667,4.0,1.0,9.0
4,1020,Warmup,-0.742380,3.666667,0.0,0.0,0.0
...,...,...,...,...,...,...,...
269,10103,ExpBlock2,-0.026413,3.666667,1.0,1.0,3.0
270,10103,Warmup,-1.405215,3.333333,1.0,2.0,4.0
271,10104,ExpBlock1,-0.531224,3.000000,0.0,0.0,0.0
272,10104,ExpBlock2,-1.023464,4.000000,0.0,1.0,1.0


## Export task scores

In [12]:
dfTeamBlockOutcomes.to_csv(OUTPUTDIR + "team_block_outcomes.csv", index=False)
print("%steam_block_outcomes.csv generated." % (OUTPUTDIR))


/Users/angelina/Dropbox (Emotive Computing)/CPS_Gaze_Fixations/RQA/team_block_outcomes.csv generated.
